# Federated Learning avec Secure Aggregation Complète et RSA

## Fonctionnalités implémentées:

1. **Transmission des mises à jour (Δw)** au lieu des poids complets
2. **Signature RSA réelle** pour l'authentification des clients
3. **Secure Aggregation avec:**
   - Protocole Diffie-Hellman pour génération de clés partagées
   - Masquage pairwise (chaque client masque avec tous les autres)
   - SHA256 pour dériver des graines (seeds) déterministes
   - Génération de masques aléatoires déterministes via PRG
4. **Vérification des signatures** côté serveur avant agrégation

In [57]:
# =============================================================================
# IMPORTS
# =============================================================================

import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
tf.get_logger().setLevel('ERROR')
tf.debugging.set_log_device_placement(False)

from sklearn.metrics import f1_score, accuracy_score
import warnings
warnings.filterwarnings('ignore')

# Cryptographie
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.backends import default_backend
import hashlib
import pickle

# Seeds
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
import random
random.seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)

print(" Imports réussis")

 Imports réussis


## 1. Classe RSA pour l'Authentification

In [58]:
# =============================================================================
# CLASSE RSA POUR AUTHENTIFICATION
# =============================================================================

class RSAManager:
    """
    Gestionnaire de clés RSA pour l'authentification des clients.
    Chaque client possède une paire de clés (publique/privée).
    """
    
    def __init__(self):
        self.client_keys = {}  # {client_id: {'private': key, 'public': key}}
    
    def generate_keypair(self, client_id):
        """Génère une paire de clés RSA 2048 bits pour un client."""
        private_key = rsa.generate_private_key(
            public_exponent=65537,
            key_size=2048,
            backend=default_backend()
        )
        public_key = private_key.public_key()
        
        self.client_keys[client_id] = {
            'private': private_key,
            'public': public_key
        }
        
        return private_key, public_key
    
    def get_private_key(self, client_id):
        """Récupère la clé privée d'un client."""
        return self.client_keys[client_id]['private']
    
    def get_public_key(self, client_id):
        """Récupère la clé publique d'un client."""
        return self.client_keys[client_id]['public']
    
    def sign_data(self, data, private_key):
        """
        Signe des données avec une clé privée RSA.
        Utilise PSS padding et SHA256.
        
        Args:
            data: Données à signer (list of numpy arrays)
            private_key: Clé privée RSA
        
        Returns:
            signature: bytes
        """
        # Sérialiser les données
        data_bytes = pickle.dumps([arr.tolist() for arr in data])
        
        # Hasher les données
        data_hash = hashlib.sha256(data_bytes).digest()
        
        # Signer le hash
        signature = private_key.sign(
            data_hash,
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.MAX_LENGTH
            ),
            hashes.SHA256()
        )
        
        return signature
    
    def verify_signature(self, data, signature, public_key):
        """
        Vérifie une signature RSA.
        
        Args:
            data: Données originales (list of numpy arrays)
            signature: Signature à vérifier
            public_key: Clé publique RSA
        
        Returns:
            bool: True si signature valide, False sinon
        """
        try:
            # Recalculer le hash des données
            data_bytes = pickle.dumps([arr.tolist() for arr in data])
            data_hash = hashlib.sha256(data_bytes).digest()
            
            # Vérifier la signature
            public_key.verify(
                signature,
                data_hash,
                padding.PSS(
                    mgf=padding.MGF1(hashes.SHA256()),
                    salt_length=padding.PSS.MAX_LENGTH
                ),
                hashes.SHA256()
            )
            return True
        except Exception:
            return False

print(" Classe RSAManager définie")

 Classe RSAManager définie


## 2. Classe Secure Aggregation (Diffie-Hellman + Masquage Pairwise)

In [59]:
# =============================================================================
# CLASSE SECURE AGGREGATION
# =============================================================================

class SecureAggregation:
    """
    Implémentation de Secure Aggregation avec:
    - Diffie-Hellman pour l'échange de clés
    - SHA256 pour dériver des seeds
    - Masquage pairwise déterministe
    """
    
    def __init__(self, num_clients):
        self.num_clients = num_clients
        # Paramètres Diffie-Hellman (nombre premier sûr)
        self.p = 2**61 - 1  # Nombre premier de Mersenne (suffisant pour démo)
        self.g = 2  # Générateur
        
        # Clés privées DH pour chaque client
        self.private_keys = {}
        # Clés publiques DH pour chaque client
        self.public_keys = {}
        
        # Graines partagées (seeds) entre paires de clients
        self.shared_seeds = {}
    
    def generate_dh_keypair(self, client_id):
        """
        Génère une paire de clés Diffie-Hellman pour un client.
        
        Returns:
            (private_key, public_key)
        """
        # Clé privée: nombre aléatoire (utiliser secrets pour les grands nombres)
        import secrets
        private_key = secrets.randbelow(self.p - 2) + 2
        # Clé publique: g^private mod p
        public_key = pow(self.g, private_key, self.p)
        
        self.private_keys[client_id] = private_key
        self.public_keys[client_id] = public_key
        
        return private_key, public_key
    
    def compute_shared_seed(self, client_i, client_j, public_key_j):
        """
        Calcule la graine partagée entre deux clients via Diffie-Hellman.
        
        Args:
            client_i: ID du client local
            client_j: ID du client distant
            public_key_j: Clé publique DH du client j
        
        Returns:
            seed: int (graine dérivée via SHA256)
        """
        # Secret partagé DH: (g^b)^a = g^(ab) mod p
        private_key_i = self.private_keys[client_i]
        shared_secret = pow(public_key_j, private_key_i, self.p)
        
        # Dériver une graine via SHA256
        shared_bytes = str(shared_secret).encode()
        seed_hash = hashlib.sha256(shared_bytes).digest()
        # Convertir en entier pour seed numpy
        seed = int.from_bytes(seed_hash[:4], byteorder='big')
        
        return seed
    
    def generate_pairwise_mask(self, seed, shape):
        """
        Génère un masque aléatoire déterministe à partir d'une graine.
        
        Args:
            seed: Graine pour le générateur aléatoire
            shape: Forme du masque
        
        Returns:
            mask: numpy array
        """
        # Créer un générateur avec la graine
        rng = np.random.RandomState(seed)
        # Masque aléatoire gaussien
        mask = rng.randn(*shape) * 0.1
        return mask
    
    def mask_weights(self, client_id, weight_updates):
        """
        Masque les mises à jour de poids avec masquage pairwise.
        
        Pour chaque autre client j:
        - Si i < j: ajoute le masque +M_ij
        - Si i > j: ajoute le masque -M_ij
        
        Ainsi, lors de l'agrégation, les masques s'annulent:
        M_ij - M_ij = 0
        
        Args:
            client_id: ID du client
            weight_updates: Liste des mises à jour de poids (Δw)
        
        Returns:
            masked_updates: Liste des mises à jour masquées
        """
        masked_updates = [w.copy() for w in weight_updates]
        
        # Pour chaque autre client
        for other_id in range(self.num_clients):
            if other_id == client_id:
                continue
            
            # Récupérer la clé publique de l'autre client
            public_key_other = self.public_keys[other_id]
            
            # Calculer la graine partagée
            seed = self.compute_shared_seed(client_id, other_id, public_key_other)
            
            # Générer et appliquer les masques pour chaque couche
            for layer_idx, update in enumerate(masked_updates):
                mask = self.generate_pairwise_mask(seed + layer_idx, update.shape)
                
                # Ajouter ou soustraire selon l'ordre des IDs
                if client_id < other_id:
                    masked_updates[layer_idx] += mask
                else:
                    masked_updates[layer_idx] -= mask
        
        return masked_updates

print(" Classe SecureAggregation définie")

 Classe SecureAggregation définie


## 3. Configuration

In [60]:
# =============================================================================
# CONFIGURATION
# =============================================================================

NUM_CLIENTS = 20
NUM_ROUNDS = 5
LOCAL_EPOCHS = 1 
BATCH_SIZE = 32

## 4. Chargement des Données

In [61]:
# =============================================================================
# CHARGEMENT DES DONNÉES MNIST
# =============================================================================

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Normalisation
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

# Reshape
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

# One-hot encoding
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)
y_test_classes = np.argmax(y_test, axis=1)

print(f" Données chargées:")
print(f"  - Train: {x_train.shape}")
print(f"  - Test: {x_test.shape}")

 Données chargées:
  - Train: (60000, 28, 28, 1)
  - Test: (10000, 28, 28, 1)


## 5. Partitionnement des Données (IID)

In [62]:
# =============================================================================
# PARTITIONNEMENT DES DONNÉES
# =============================================================================

def create_iid_clients(x_train, y_train, num_clients):
    """Distribue les données de manière IID entre les clients."""
    indices = np.random.permutation(len(x_train))
    samples_per_client = len(x_train) // num_clients
    
    client_data = []
    for i in range(num_clients):
        start_idx = i * samples_per_client
        end_idx = start_idx + samples_per_client
        
        client_indices = indices[start_idx:end_idx]
        client_x = x_train[client_indices]
        client_y = y_train[client_indices]
        
        client_data.append((client_x, client_y))
    
    return client_data

client_datasets = create_iid_clients(x_train, y_train, NUM_CLIENTS)
print(f" Données distribuées à {NUM_CLIENTS} clients")
print(f"  - Échantillons par client: {len(client_datasets[0][0])}")

 Données distribuées à 20 clients
  - Échantillons par client: 3000


## 6. Définition du Modèle

In [63]:
# =============================================================================
# MODÈLE CNN
# =============================================================================

def create_cnn_model():
    """Crée un modèle CNN simple pour MNIST."""
    model = keras.Sequential([
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu", input_shape=(28, 28, 1)),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(10, activation="softmax")
    ])
    
    model.compile(
        loss="categorical_crossentropy",
        optimizer="adam",
        metrics=["accuracy"]
    )
    
    return model

print(" Architecture du modèle définie")

 Architecture du modèle définie


## 7. Classe Client FL

In [64]:
# =============================================================================
# CLASSE CLIENT
# =============================================================================

class FederatedClient:
    """
    Client de Federated Learning avec:
    - Entraînement local
    - Calcul des mises à jour (Δw)
    - Masquage des mises à jour
    - Signature RSA
    """
    
    def __init__(self, client_id, data, rsa_manager, sec_agg):
        self.client_id = client_id
        self.x_train, self.y_train = data
        self.rsa_manager = rsa_manager
        self.sec_agg = sec_agg
        
        # Modèle local
        self.model = create_cnn_model()
    
    def set_weights(self, weights):
        """Reçoit les poids globaux du serveur."""
        self.model.set_weights(weights)
    
    def get_weights(self):
        """Retourne les poids actuels."""
        return self.model.get_weights()
    
    def train(self, epochs=1, batch_size=32, verbose=0):
        """Entraîne le modèle localement."""
        self.model.fit(
            self.x_train, self.y_train,
            batch_size=batch_size,
            epochs=epochs,
            verbose=verbose
        )
    
    def compute_weight_update(self, global_weights):
        """
        Calcule la mise à jour des poids: Δw = w_local - w_global
        
        Args:
            global_weights: Poids globaux du serveur
        
        Returns:
            weight_update: Liste de numpy arrays (Δw)
        """
        local_weights = self.get_weights()
        weight_update = [
            local_w - global_w 
            for local_w, global_w in zip(local_weights, global_weights)
        ]
        return weight_update
    
    def prepare_update(self, global_weights):
        """
        Prépare la mise à jour complète:
        1. Calcule Δw
        2. Masque Δw
        3. Signe Δw masqué
        
        Returns:
            (masked_update, signature)
        """
        # 1. Calculer Δw
        weight_update = self.compute_weight_update(global_weights)
        
        # 2. Masquer Δw
        masked_update = self.sec_agg.mask_weights(self.client_id, weight_update)
        
        # 3. Signer Δw masqué
        private_key = self.rsa_manager.get_private_key(self.client_id)
        signature = self.rsa_manager.sign_data(masked_update, private_key)
        
        return masked_update, signature

print(" Classe FederatedClient définie")

 Classe FederatedClient définie


## 8. Classe Serveur FL

In [65]:
# =============================================================================
# CLASSE SERVEUR
# =============================================================================

class FederatedServer:
    """
    Serveur de Federated Learning avec:
    - Vérification des signatures RSA
    - Agrégation des mises à jour masquées
    - Mise à jour du modèle global
    """
    
    def __init__(self, rsa_manager):
        self.rsa_manager = rsa_manager
        self.model = create_cnn_model()
    
    def get_weights(self):
        """Retourne les poids globaux."""
        return self.model.get_weights()
    
    def set_weights(self, weights):
        """Met à jour les poids globaux."""
        self.model.set_weights(weights)
    
    def verify_and_aggregate(self, client_updates):
        """
        Vérifie les signatures et agrège les mises à jour.
        
        Args:
            client_updates: Liste de tuples (client_id, masked_update, signature)
        
        Returns:
            aggregated_update: Moyenne des mises à jour vérifiées
        """
        verified_updates = []
        
        print("  Vérification des signatures:")
        for client_id, masked_update, signature in client_updates:
            # Récupérer la clé publique du client
            public_key = self.rsa_manager.get_public_key(client_id)
            
            # Vérifier la signature
            is_valid = self.rsa_manager.verify_signature(
                masked_update, signature, public_key
            )
            
            if is_valid:
                print(f"     Client {client_id}: Signature valide")
                verified_updates.append(masked_update)
            else:
                print(f"     Client {client_id}: Signature INVALIDE - Rejeté")
        
        if len(verified_updates) == 0:
            raise ValueError("Aucune mise à jour valide!")
        
        # Agrégation: moyenne des mises à jour
        # Les masques pairwise s'annulent automatiquement!
        aggregated_update = [
            np.mean([update[i] for update in verified_updates], axis=0)
            for i in range(len(verified_updates[0]))
        ]
        
        print(f"   {len(verified_updates)}/{len(client_updates)} mises à jour agrégées")
        
        return aggregated_update
    
    def update_global_model(self, aggregated_update):
        """
        Met à jour le modèle global: w_new = w_old + Δw_avg
        
        Args:
            aggregated_update: Mise à jour agrégée
        """
        current_weights = self.get_weights()
        new_weights = [
            current_w + update
            for current_w, update in zip(current_weights, aggregated_update)
        ]
        self.set_weights(new_weights)
    
    def evaluate(self, x_test, y_test):
        """Évalue le modèle global."""
        loss, accuracy = self.model.evaluate(x_test, y_test, verbose=0)
        return loss, accuracy

print(" Classe FederatedServer définie")

 Classe FederatedServer définie


## 9. Baselines de Comparaison

Avant le Federated Learning, nous allons établir deux baselines:
1. **Apprentissage Centralisé** (100% des données)
2. **Un seul client** (1/N des données)

In [66]:
# =============================================================================
# BASELINE 1: APPRENTISSAGE CENTRALISÉ (100% DES DONNÉES)
# =============================================================================

print("\n" + "="*70)
print("BASELINE 1: APPRENTISSAGE CENTRALISÉ")
print("="*70)
print(f"Entraînement avec 100% des données ({len(x_train)} échantillons)\n")

# Créer et entraîner le modèle centralisé
model_centralized = create_cnn_model()

print("Entraînement en cours...")
history_centralized = model_centralized.fit(
    x_train, y_train,
    batch_size=BATCH_SIZE,
    epochs=NUM_ROUNDS,
    validation_split=0.1,
    verbose=1
)

# Évaluation
print("\nÉvaluation sur le test set...")
y_pred_centralized = model_centralized.predict(x_test, verbose=0)
y_pred_centralized_classes = np.argmax(y_pred_centralized, axis=1)

accuracy_centralized = accuracy_score(y_test_classes, y_pred_centralized_classes)
f1_centralized = f1_score(y_test_classes, y_pred_centralized_classes, average='weighted')

print(f"\n Résultats - Centralisé:")
print(f"  Accuracy: {accuracy_centralized:.4f}")
print(f"  F1-Score: {f1_centralized:.4f}")

# Historique
print(f"\nHistorique d'entraînement:")
for epoch in range(NUM_ROUNDS):
    print(f"  Epoch {epoch+1}: "
          f"Loss = {history_centralized.history['loss'][epoch]:.4f}, "
          f"Accuracy = {history_centralized.history['accuracy'][epoch]:.4f}")

print("\n" + "="*70)


BASELINE 1: APPRENTISSAGE CENTRALISÉ
Entraînement avec 100% des données (60000 échantillons)

Entraînement en cours...
Epoch 1/5
1688/1688 [==============================] - 12s 7ms/step - loss: 0.2193 - accuracy: 0.9327 - val_loss: 0.0563 - val_accuracy: 0.9827
Epoch 2/5
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0836 - accuracy: 0.9745 - val_loss: 0.0431 - val_accuracy: 0.9870
Epoch 3/5
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0664 - accuracy: 0.9794 - val_loss: 0.0377 - val_accuracy: 0.9902
Epoch 4/5
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0554 - accuracy: 0.9825 - val_loss: 0.0357 - val_accuracy: 0.9893
Epoch 5/5
1688/1688 [==============================] - 9s 6ms/step - loss: 0.0517 - accuracy: 0.9834 - val_loss: 0.0318 - val_accuracy: 0.9910

Évaluation sur le test set...

 Résultats - Centralisé:
  Accuracy: 0.9902
  F1-Score: 0.9902

Historique d'entraînement:
  Epoch 1: Loss = 0.2193, Accuracy = 0.93

In [67]:
# =============================================================================
# BASELINE 2: UN SEUL CLIENT
# =============================================================================

print("\n" + "="*70)
print("BASELINE 2: UN SEUL CLIENT")
print("="*70)
print(f"Entraînement avec 1/{NUM_CLIENTS} des données ({len(client_datasets[0][0])} échantillons)\n")

# Créer et entraîner le modèle avec les données d'un seul client
model_single = create_cnn_model()
single_client_x, single_client_y = client_datasets[0]

print("Entraînement en cours...")
history_single = model_single.fit(
    single_client_x, single_client_y,
    batch_size=BATCH_SIZE,
    epochs=NUM_ROUNDS,
    validation_split=0.1,
    verbose=1
)

# Évaluation
print("\nÉvaluation sur le test set...")
y_pred_single = model_single.predict(x_test, verbose=0)
y_pred_single_classes = np.argmax(y_pred_single, axis=1)

accuracy_single = accuracy_score(y_test_classes, y_pred_single_classes)
f1_single = f1_score(y_test_classes, y_pred_single_classes, average='weighted')

print(f"\n Résultats - Client unique:")
print(f"  Accuracy: {accuracy_single:.4f}")
print(f"  F1-Score: {f1_single:.4f}")

# Historique
print(f"\nHistorique d'entraînement:")
for epoch in range(NUM_ROUNDS):
    print(f"  Epoch {epoch+1}: "
          f"Loss = {history_single.history['loss'][epoch]:.4f}, "
          f"Accuracy = {history_single.history['accuracy'][epoch]:.4f}")

print("\n" + "="*70)


BASELINE 2: UN SEUL CLIENT
Entraînement avec 1/20 des données (3000 échantillons)

Entraînement en cours...
Epoch 1/5
85/85 [==============================] - 1s 11ms/step - loss: 1.3269 - accuracy: 0.5767 - val_loss: 0.4765 - val_accuracy: 0.8500
Epoch 2/5
85/85 [==============================] - 1s 7ms/step - loss: 0.4376 - accuracy: 0.8619 - val_loss: 0.2557 - val_accuracy: 0.9300
Epoch 3/5
85/85 [==============================] - 1s 6ms/step - loss: 0.2786 - accuracy: 0.9148 - val_loss: 0.1812 - val_accuracy: 0.9567
Epoch 4/5
85/85 [==============================] - 0s 6ms/step - loss: 0.2241 - accuracy: 0.9319 - val_loss: 0.1458 - val_accuracy: 0.9533
Epoch 5/5
85/85 [==============================] - 1s 6ms/step - loss: 0.1816 - accuracy: 0.9463 - val_loss: 0.1243 - val_accuracy: 0.9533

Évaluation sur le test set...

 Résultats - Client unique:
  Accuracy: 0.9555
  F1-Score: 0.9555

Historique d'entraînement:
  Epoch 1: Loss = 1.3269, Accuracy = 0.5767
  Epoch 2: Loss = 0.4376,

## 10. Initialisation du Système Fédéré

In [68]:
# =============================================================================
# INITIALISATION DU SYSTÈME FL
# =============================================================================

print("\n" + "="*70)
print("INITIALISATION DU SYSTÈME")
print("="*70)

# 1. Créer les gestionnaires
rsa_manager = RSAManager()
sec_agg = SecureAggregation(NUM_CLIENTS)

print("\n1. Génération des clés RSA pour chaque client...")
for client_id in range(NUM_CLIENTS):
    rsa_manager.generate_keypair(client_id)
    print(f"   Client {client_id}: Clés RSA générées")

print("\n2. Génération des clés Diffie-Hellman pour chaque client...")
for client_id in range(NUM_CLIENTS):
    sec_agg.generate_dh_keypair(client_id)
    print(f"   Client {client_id}: Clés DH générées")

# 2. Créer les clients
print("\n3. Création des clients...")
clients = []
for client_id in range(NUM_CLIENTS):
    client = FederatedClient(
        client_id=client_id,
        data=client_datasets[client_id],
        rsa_manager=rsa_manager,
        sec_agg=sec_agg
    )
    clients.append(client)
    print(f"   Client {client_id} créé")

# 3. Créer le serveur
print("\n4. Création du serveur...")
server = FederatedServer(rsa_manager)
print("   Serveur créé")

print("\n" + "="*70)
print(" SYSTÈME INITIALISÉ")
print("="*70)


INITIALISATION DU SYSTÈME

1. Génération des clés RSA pour chaque client...
   Client 0: Clés RSA générées
   Client 1: Clés RSA générées
   Client 2: Clés RSA générées
   Client 3: Clés RSA générées
   Client 4: Clés RSA générées
   Client 5: Clés RSA générées
   Client 6: Clés RSA générées
   Client 7: Clés RSA générées
   Client 8: Clés RSA générées
   Client 9: Clés RSA générées
   Client 10: Clés RSA générées
   Client 11: Clés RSA générées
   Client 12: Clés RSA générées
   Client 13: Clés RSA générées
   Client 14: Clés RSA générées
   Client 15: Clés RSA générées
   Client 16: Clés RSA générées
   Client 17: Clés RSA générées
   Client 18: Clés RSA générées
   Client 19: Clés RSA générées

2. Génération des clés Diffie-Hellman pour chaque client...
   Client 0: Clés DH générées
   Client 1: Clés DH générées
   Client 2: Clés DH générées
   Client 3: Clés DH générées
   Client 4: Clés DH générées
   Client 5: Clés DH générées
   Client 6: Clés DH générées
   Client 7: Clés DH g

## 11. Entraînement Fédéré avec Secure Aggregation

In [69]:
# =============================================================================
# ENTRAÎNEMENT FÉDÉRÉ
# =============================================================================

print("\n" + "="*70)
print("DÉMARRAGE DE L'ENTRAÎNEMENT FÉDÉRÉ")
print("="*70)

history = {
    'round': [],
    'loss': [],
    'accuracy': []
}

for round_num in range(NUM_ROUNDS):
    print(f"\n{'='*70}")
    print(f"ROUND {round_num + 1}/{NUM_ROUNDS}")
    print(f"{'='*70}")
    
    # 1. Distribuer les poids globaux
    print("\n Distribution des poids globaux...")
    global_weights = server.get_weights()
    for client in clients:
        client.set_weights(global_weights)
    print("   Poids distribués à tous les clients")
    
    # 2. Entraînement local
    print("\n Entraînement local...")
    for client in clients:
        client.train(epochs=LOCAL_EPOCHS, batch_size=BATCH_SIZE, verbose=0)
    print(f"   {NUM_CLIENTS} clients entraînés")
    
    # 3. Préparation des mises à jour (calcul Δw + masquage + signature)
    print("\n Préparation des mises à jour (Δw masqué + signature)...")
    client_updates = []
    for client in clients:
        masked_update, signature = client.prepare_update(global_weights)
        client_updates.append((client.client_id, masked_update, signature))
    print(f"   {len(client_updates)} mises à jour préparées")
    
    # 4. Agrégation sécurisée côté serveur
    print("\n Agrégation sécurisée sur le serveur...")
    aggregated_update = server.verify_and_aggregate(client_updates)
    
    # 5. Mise à jour du modèle global
    print("\n Mise à jour du modèle global...")
    server.update_global_model(aggregated_update)
    print("   Modèle global mis à jour")
    
    # 6. Évaluation
    print("\n Évaluation...")
    loss, accuracy = server.evaluate(x_test, y_test)
    print(f"  Loss: {loss:.4f}")
    print(f"  Accuracy: {accuracy:.4f}")
    
    history['round'].append(round_num + 1)
    history['loss'].append(loss)
    history['accuracy'].append(accuracy)

print("\n" + "="*70)
print(" ENTRAÎNEMENT TERMINÉ")
print("="*70)


DÉMARRAGE DE L'ENTRAÎNEMENT FÉDÉRÉ

ROUND 1/5

 Distribution des poids globaux...
   Poids distribués à tous les clients

 Entraînement local...
   20 clients entraînés

 Préparation des mises à jour (Δw masqué + signature)...
   20 mises à jour préparées

 Agrégation sécurisée sur le serveur...
  Vérification des signatures:
     Client 0: Signature valide
     Client 1: Signature valide
     Client 2: Signature valide
     Client 3: Signature valide
     Client 4: Signature valide
     Client 5: Signature valide
     Client 6: Signature valide
     Client 7: Signature valide
     Client 8: Signature valide
     Client 9: Signature valide
     Client 10: Signature valide
     Client 11: Signature valide
     Client 12: Signature valide
     Client 13: Signature valide
     Client 14: Signature valide
     Client 15: Signature valide
     Client 16: Signature valide
     Client 17: Signature valide
     Client 18: Signature valide
     Client 19: Signature valide
   20/20 mises à jour

## 12. Résultats Finaux du FL

In [70]:
# =============================================================================
# RÉSULTATS FINAUX
# =============================================================================

print("\n" + "="*70)
print("RÉSULTATS FINAUX")
print("="*70)

# Prédictions finales
y_pred = server.model.predict(x_test, verbose=0)
y_pred_classes = np.argmax(y_pred, axis=1)

# Métriques
final_accuracy = accuracy_score(y_test_classes, y_pred_classes)
final_f1 = f1_score(y_test_classes, y_pred_classes, average='weighted')

print(f"\nMétriques finales:")
print(f"  Accuracy: {final_accuracy:.4f}")
print(f"  F1-Score: {final_f1:.4f}")

# Historique
print(f"\nHistorique d'entraînement:")
for i in range(len(history['round'])):
    print(f"  Round {history['round'][i]}: "
          f"Loss = {history['loss'][i]:.4f}, "
          f"Accuracy = {history['accuracy'][i]:.4f}")

print("\n" + "="*70)


RÉSULTATS FINAUX

Métriques finales:
  Accuracy: 0.9671
  F1-Score: 0.9671

Historique d'entraînement:
  Round 1: Loss = 0.3607, Accuracy = 0.9113
  Round 2: Loss = 0.1992, Accuracy = 0.9459
  Round 3: Loss = 0.1521, Accuracy = 0.9570
  Round 4: Loss = 0.1260, Accuracy = 0.9630
  Round 5: Loss = 0.1107, Accuracy = 0.9671



## 13. Vérification du Masquage Pairwise

In [71]:
# =============================================================================
# VÉRIFICATION QUE LES MASQUES S'ANNULENT
# =============================================================================

print("\n" + "="*70)
print("VÉRIFICATION DU MASQUAGE PAIRWISE")
print("="*70)

print("\nTest: Les masques pairwise doivent s'annuler lors de l'agrégation")
print("\nCréation de mises à jour fictives...")

# Créer des mises à jour fictives (zéros)
dummy_shape = (10, 10)  # Forme simplifiée
dummy_updates = [np.zeros(dummy_shape) for _ in range(NUM_CLIENTS)]

# Appliquer les masques
print("Application des masques pairwise...")
masked_updates = []
for client_id in range(NUM_CLIENTS):
    masked = sec_agg.mask_weights(client_id, [dummy_updates[client_id]])
    masked_updates.append(masked[0])

# Agréger
print("Agrégation des mises à jour masquées...")
aggregated = np.mean(masked_updates, axis=0)

# Vérifier
max_deviation = np.max(np.abs(aggregated))
print(f"\nRésultat:")
print(f"  Déviation maximale de zéro: {max_deviation:.10f}")

if max_deviation < 1e-6:
    print(f"   SUCCÈS: Les masques se sont correctement annulés!")
else:
    print(f"   ERREUR: Les masques ne s'annulent pas parfaitement")

print("\n" + "="*70)


VÉRIFICATION DU MASQUAGE PAIRWISE

Test: Les masques pairwise doivent s'annuler lors de l'agrégation

Création de mises à jour fictives...
Application des masques pairwise...
Agrégation des mises à jour masquées...

Résultat:
  Déviation maximale de zéro: 0.0000000000
   SUCCÈS: Les masques se sont correctement annulés!



## 14. Comparaison des Résultats

In [72]:
# =============================================================================
# COMPARAISON FINALE DES TROIS APPROCHES
# =============================================================================

print("\n" + "="*70)
print("COMPARAISON FINALE DES TROIS APPROCHES")
print("="*70)

# Calculer les résultats du FL
print("\nCalcul des métriques pour Federated Learning...")
y_pred_fl = server.model.predict(x_test, verbose=0)
y_pred_fl_classes = np.argmax(y_pred_fl, axis=1)
accuracy_fl = accuracy_score(y_test_classes, y_pred_fl_classes)
f1_fl = f1_score(y_test_classes, y_pred_fl_classes, average='weighted')

# Tableau de comparaison
print("\n" + "="*70)
print("TABLEAU COMPARATIF")
print("="*70)
print(f"\n{'Approche':<35} {'Données':<15} {'Accuracy':<12} {'F1-Score':<12}")
print("-"*70)
print(f"{'1. Centralisé':<35} {'100%':<15} {accuracy_centralized:<12.4f} {f1_centralized:<12.4f}")
print(f"{'2. Client unique':<35} {f'1/{NUM_CLIENTS} = {100/NUM_CLIENTS:.1f}%':<15} {accuracy_single:<12.4f} {f1_single:<12.4f}")
print(f"{'3. Federated Learning':<35} {f'{NUM_CLIENTS} clients':<15} {accuracy_fl:<12.4f} {f1_fl:<12.4f}")
print("-"*70)


COMPARAISON FINALE DES TROIS APPROCHES

Calcul des métriques pour Federated Learning...

TABLEAU COMPARATIF

Approche                            Données         Accuracy     F1-Score    
----------------------------------------------------------------------
1. Centralisé                       100%            0.9902       0.9902      
2. Client unique                    1/20 = 5.0%     0.9555       0.9555      
3. Federated Learning               20 clients      0.9671       0.9671      
----------------------------------------------------------------------
